In [1]:
import os
import json 
import ollama
import gradio as gr

f:\LLM engineering\llmenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
MODEL = "llama3.2:1b"

In [3]:
system_msg = "you are a helpful assistant for an Airline all flightai"
system_msg += "Give short, courtous answer, no more than 30 words"
system_msg += "Always be accurate. If you dont know the answer, say so"

In [4]:
def chat(message, history):
    messages = [{"role": "system", "content": system_msg}]
    
    for user_message, assistant_message in history:
        messages.append({"role": "user", "content": user_message})
        messages.append({"role": "assistant", "content": assistant_message})
    
    messages.append({"role": "user", "content": message})  

    response = ollama.chat(model=MODEL, messages=messages)
     
    return response['message']['content']


In [5]:
gr.ChatInterface(fn = chat).launch()

f:\LLM engineering\llmenv\Lib\site-packages\gradio\chat_interface.py:334: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


In [12]:

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [13]:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [14]:
tools = [{"type": "function", "function": price_function}]

In [15]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response, city

In [20]:
def chat(message, history):
    messages = [{"role": "system", "content": system_msg}] + history + [{"role": "user", "content": message}]
    
    response = ollama.chat(model=MODEL, messages=messages, tools=tools)

    # Ensure response structure is correct
    if isinstance(response, dict) and "message" in response:
        if response.get("choices", [{}])[0].get("finish_reason") == "tool_calls":
            message = response["message"]
            response, city = handle_tool_call(message)
            
            messages.append(message)
            messages.append(response)
            
            response = ollama.chat(model=MODEL, messages=messages)

    return response['message']['content']


In [21]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.
